<h1 style="text-align:center;"> Google Code Golf 2025 Submission </h1>

## Change log

1. Decoder Function
Created a decoder function to recover the initial code in a readable format. This makes it easier to understand the original logic before focusing on optimization. (Think of it as a hint or reference point for future improvements.)

2. Optimization Hints
Added several hints about potential optimization strategies. These may be useful for anyone looking to refine their code further.

3. Code Optimizations
Improved several solutions to make them shorter and cleaner. While these are not as compact as the top “gold” solutions in [this document](https://docs.google.com/spreadsheets/u/1/d/e/2PACX-1vQ7RUqwrtwRD2EJbgMRrccAHkwUQZgFe2fsROCR1WV5LA1naxL0pU2grjQpcWC2HU3chdGwIOUpeuoK/pubhtml#gid=1427788625), they are significantly better than the previous versions.


## Optimized code hints

The goal is to simplify logic, remove unnecessary steps, and reduce code length 
while keeping correctness.

Main observations:
1. Many solutions generated by LLMs include redundant conditions, often checking edge cases and returning the initial grid. In most tasks, this is unnecessary and can be removed, saving ~20–30 characters.
2. Extra code is sometimes added at the beginning to compute the grid’s length and width. For square grids or fixed-size grids, these calculations can be replaced or omitted, reducing code length by ~5–30 characters.
3. For functions like `range` and `enumerate`, it is sometimes more efficient to keep the full names rather than aliasing them. In some cases, the uncompressed version with full keywords is actually shorter.
4. Using operators such as `//` or `%` can further reduce length, often saving another ~10–20 characters.

These strategies aim to produce shorter, cleaner code.

## Respect, Credit, and Recognition

Huge thanks to everyone for sharing and contributing in this competition!

Much of my reading and code inspiration comes mainly from LLMs and the amazing work of @jazivxt, @garrymoss, @cheeseexports, @tonylica.

Also,credit goes to the many contributors—even if I didn’t reference code from them directly:
@adilshamim8 @adyanthm @boristown @bibanh @cheeseexports @daosyduyminh @dedquoc @fedimser @henrychibueze @jacekwl @jaejohn @jazivxt @jeroencottaar @katsuyanomura @kosirowada @krishnayadav456wrsty @kuntalmaity @limalkasadith @mmoffitt @mpwolke @muhammaddanyalmalik @muhammadqasimshabbir @nina2025 @ottitsch @quannguyn12 @raviannaswamy @seshura @seshurajup @shevtsovapolina @taylorsamarel @zshashz

This notebook compresses and submits our optimized task solutions using zlib compression to minimize file sizes for the competition.

In [ ]:
import zipfile, json, os, copy
import sys
sys.path.append("/kaggle/input/google-code-golf-2025/code_golf_utils")
from code_golf_utils import *

source = "/kaggle/input/neurips-code-golf-2025-optim"

## Unzip code 

To udnertand the code better and potential optimize it, this part unzip those soltuion that has been zipped previously

In [ ]:
import re, ast, zlib
def unzip_src(wrapper: bytes) -> bytes:
    """
    Accepts the exact bytes returned by zip_src(...) and returns the original src bytes.
    """
    s = wrapper.decode('latin-1', errors='strict')
    m = re.search(r'bytes\((?P<q>"""|")', s, flags=re.S)
    if not m:
        raise ValueError("Could not find compressed literal.")
    q = m.group('q')
    i = m.end()
    j = s.find(q, i)
    if j < 0:
        raise ValueError("Unterminated string literal.")
    # Evaluate the literal to interpret escapes like \x00 and \\ correctly
    lit = ast.literal_eval(q + s[i:j] + q)      # -> Python str with actual chars (incl. \x00)
    comp = lit.encode('latin-1', errors='strict')  # map codepoints 0–255 back to bytes
    return zlib.decompress(comp)

# Optional: if you already extracted the inner literal text (without quotes) from the wrapper:
def unzip_literal_text(literal_text: str) -> bytes:
    """
    Given the inner text between the quotes (with escapes like \\x00), return original src bytes.
    """
    s = ast.literal_eval('"' + literal_text + '"')
    return zlib.decompress(s.encode('latin-1'))

In [ ]:
unzipped_solutions_path = "/kaggle/working/unzipped" 
os.makedirs(unzipped_solutions_path, exist_ok=True)

for task_num in range(1, 401):
    path_in  = f"{source}/task{task_num:03d}.py"
    path_out = f"{unzipped_solutions_path}/task{task_num:03d}.py"

    with open(path_in, "rb") as f: s = f.read()
    with open(path_out, "wb") as o:
        try:
            o.write(unzip_src(s))
        except Exception:
            o.write(s)

## Create sumbiission

Use zipper function for each task

Create final zip archive

In [ ]:
from zlib import compress

def zip_src(src):
    compression_level = 9  # Max Compression
    
    # We prefer that compressed source not end in a quotation mark
    while (compressed := compress(src, compression_level))[-1] == ord('"'): 
        src += b"#"
    
    def sanitize(b_in):
        """Clean up problematic bytes in compressed b-string"""
        b_out = bytearray()
        for b in b_in:
            if b == 0:         
                b_out += b"\\x00"
            elif b == ord("\r"): 
                b_out += b"\\r"
            elif b == ord("\\"): 
                b_out += b"\\\\"
            else: 
                b_out.append(b)
        return b"" + b_out
    
    compressed = sanitize(compressed)
    
    delim = b'"""' if ord("\n") in compressed or ord('"') in compressed else b'"'
    
    return b"#coding:L1\nimport zlib\nexec(zlib.decompress(bytes(" + \
        delim + compressed + delim + \
        b',"L1")))'


Processing and compressing task solutions:

In [ ]:
total_save = 0
processed_tasks = 0

submission = "/kaggle/working/submission"
os.makedirs(submission, exist_ok=True)

# Process tasks 1-400
for task_num in range(1, 401):
    path_in = f"{source}/task{task_num:03d}.py"
    path_out = f"{submission}/task{task_num:03d}.py"
    
    if not os.path.exists(path_in):
        continue
    
    try:
        with open(path_in, "rb") as task_in:
            task_src = task_in.read()

        # Only compress if file has content
        if len(task_src) > 0:
            zipped_src = zip_src(task_src)
            improvement = len(task_src) - len(zipped_src)
            
            # Use compressed version if it saves space
            if improvement > 0:
                task_src = zipped_src
                total_save += improvement
            
            with open(path_out, "wb") as task_out:
                task_out.write(task_src)
            
            processed_tasks += 1
    
    except Exception as e:
        print(f"Error processing task{task_num:03d}: {e}")
        continue

print(f"Processed {processed_tasks} tasks")
print(f"Saved {total_save}b with zlib compression")

Finally, create the submission zip file:

In [ ]:
import zipfile

submission_zip = f"{submission}.zip"

with zipfile.ZipFile(submission_zip, "w", zipfile.ZIP_DEFLATED) as zipf:
    task_count = 0
    for task_num in range(1, 401):
        task_id = f"{task_num:03d}"
        src_path = f"{submission}/task{task_id}.py"
        
        if os.path.exists(src_path):
            zipf.write(src_path, arcname=f"task{task_id}.py")
            task_count += 1

print(f"Created submission zip with {task_count} tasks: {submission_zip}")

# Display zip file size
zip_size = os.path.getsize(submission_zip)
print(f"Submission zip size: {zip_size:,} bytes ({zip_size/1024:.1f} KB)")